In [ ]:
# ============================================
# SAR Image Download Script for Bajo Cauca - Confluencia
# ============================================

# --------- Install & Import Dependencies ---------
!pip install rasterio
from google.colab import drive
import ee
import geemap
import os
import numpy as np
import cv2
import rasterio
from google.colab.patches import cv2_imshow

# --------- Mount Google Drive ---------
drive.mount('/content/drive')

# --------- Authenticate and Initialize Earth Engine ---------
# IMPORTANT: Replace with your own Google Cloud project ID
ee.Authenticate()
ee.Initialize(project='YOUR-GEE-PROJECT-ID')  # <-- CHANGE THIS

In [ ]:
# --------- User Parameters ---------
ZONE = 'CONFLUENCIA'
BASE_DIR = f'/content/drive/MyDrive/{ZONE}'

# Folders: source and outputs
# Folders: source and outputs
INPUT_DIR = f'{BASE_DIR}/VV_VISUAL_FILT'
CLASSIFIED_DIR = f'{BASE_DIR}/CLASSIFIED_FILT'
AVERAGE_DIR = f'{BASE_DIR}/AVERAGE_CLASSIFIED_FILT'

os.makedirs(CLASSIFIED_DIR, exist_ok=True)
os.makedirs(AVERAGE_DIR, exist_ok=True)


In [ ]:
# --------- Get image list ---------
tif_files = sorted([f for f in os.listdir(INPUT_DIR) if f.endswith('.tif')])

# --------- Read first image to define target size ---------
with rasterio.open(os.path.join(INPUT_DIR, tif_files[0])) as src:
    height, width = src.read(1).shape
    base_profile = src.profile

print(f"Reference size: {width} x {height}")

In [ ]:
# --------- 1) Classify each image with Otsu and save ---------
for tif_file in tif_files:
    with rasterio.open(os.path.join(INPUT_DIR, tif_file)) as src:
        image = src.read(1)
        image_resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

        _, binary = cv2.threshold(image_resized, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        print(f"{tif_file} | Otsu threshold: {_}")

        # Morphological open to clean noise
        kernel = np.ones((3, 3), np.uint8)
        binary_clean = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

        # Save classified
        profile = src.profile.copy()
        profile.update(dtype=rasterio.uint8, count=1)
        out_path = f'{CLASSIFIED_DIR}/{tif_file}'
        with rasterio.open(out_path, 'w', **profile) as dst:
            dst.write(binary_clean, 1)

print("✅ Step 1: All images classified and saved.")

In [ ]:
# --------- 2) Compute annual averages (grayscale only) ---------
classified_files = sorted([f for f in os.listdir(CLASSIFIED_DIR) if f.endswith('.tif')])

current_year = classified_files[0][:4]
year_stack = []

for idx, tif_file in enumerate(classified_files):
    year = tif_file[:4]

    with rasterio.open(f"{CLASSIFIED_DIR}/{tif_file}") as src:
        img = src.read(1)
        img_resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

    year_stack.append(img_resized)

    # Check if it's last file or a new year starts
    is_last = (idx == len(classified_files) - 1)
    if is_last or (classified_files[idx + 1][:4] != year):
        avg_year = np.mean(year_stack, axis=0).astype(np.uint8)

        profile.update(dtype=rasterio.uint8, count=1)
        avg_path = f"{AVERAGE_DIR}/{current_year}.tif"
        with rasterio.open(avg_path, 'w', **profile) as dst:
            dst.write(avg_year, 1)

        print(f"✔️ Annual average for {current_year} saved (grayscale)")

        # Reset for next year
        current_year = year
        year_stack = []

print("✅ Step 2: Annual grayscale averages computed and saved.")

In [ ]:
# --------- 3) Compute global average ---------
annual_files = sorted([f for f in os.listdir(AVERAGE_DIR) if f.endswith('.tif')])

global_stack = []
for tif_file in annual_files:
    with rasterio.open(f"{AVERAGE_DIR}/{tif_file}") as src:
        img = src.read(1)
        img_resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)
        global_stack.append(img_resized)

global_average = np.mean(global_stack, axis=0).astype(np.uint8)

# Save global average
profile.update(dtype=rasterio.uint8, count=1)
global_path = f"{BASE_DIR}/global_average.tif"
with rasterio.open(global_path, 'w', **profile) as dst:
    dst.write(global_average, 1)

print("✅ Step 3: Global grayscale average saved.")
